<b> Sales By Policy Type and Month: </b>
This table would contain the total sales for each policy type and each month. It would be used to analyze the performance of different policy types over time.

<b>Claims By Policy Type and Status:</b>
 This table would contain the number and amount of claims by policy type and claim status. It would be used to monitor the claims process and identify any trends or issues.

In [0]:
%sql
CREATE or Replace temp View vw_Claims_By_Policy_Type_And_Status
AS
SELECT
    policy_type,
    claim_status,
    count(*) as total_claims,
    sum(claim_amount) as total_claim_amount
from
    silverlayer.claim C 
    JOIN silverlayer.policy P 
      on C.policy_id = P.policy_id 
GROUP BY
   policy_type,
   claim_status HAVING P.policy_type is NOT NULL;

In [0]:
%sql
SELECT * from vw_Claims_By_Policy_Type_And_Status

In [0]:
%sql
MERGE into goldlayer.claims_by_policy_type_and_status AS T USING vw_Claims_By_Policy_Type_And_Status as S on T.claim_status = S.claim_status and T.policy_type = S.policy_type WHEN MATCHED THEN 
UPDATE 
SET T.total_claim_amount = S.total_claim_amount, T.total_claims = S.total_claims, T.updated_timestamp = current_timestamp()
WHEN NOT MATCHED THEN
INSERT(
    policy_type, claim_status, total_claims, total_claim_amount, updated_timestamp
)
VALUES( 
    S.policy_type, S.claim_status, S.total_claims, S.total_claim_amount, current_timestamp()
)


In [0]:
%sql
SELECT * FROM goldlayer.claims_by_policy_type_and_status;

In [0]:
#df_claims_by_policyType_Status = spark.sql("SELECT * from vw_Claims_By_Policy_Type_And_Status")
#df_claims_by_policyType_Status.write.mode("append").option("mergeSchema", "true").saveAsTable("goldlayer.Claims_By_Policy_Type_And_Status")

<b>
Analyze the claim data based on the policy type like AVG, MAX, MIN, Count of claim.

In [0]:
df_claim_analysis= spark.sql("SELECT policy_type, avg(C.claim_amount) As avg_claim_amount, max(C.claim_amount) As max_claim_amount, min(C.claim_amount) As min_claim_amount, count(DISTINCT C.claim_id) As total_claims FROM silverlayer.claim C JOIN silverlayer.policy P on C.policy_id = P.policy_id GROUP BY P.policy_type HAVING policy_type is NOT NULL")

display(df_claim_analysis)

In [0]:
from pyspark.sql.functions import *
df_merged_date_timestamp= df_claim_analysis.withColumn("updated_timestamp" , current_timestamp())
df_merged_date_timestamp.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("goldlayer.Claim_Analysis")

In [0]:
%sql
select * from goldlayer.Claim_Analysis